In [5]:
import os
import requests
from dotenv import load_dotenv
import json
load_dotenv();

In [20]:
NOTION_API_KEY = os.getenv("notion_key")
DATA_SOURCE_ID = os.getenv("data_id")

print("✅ 변수 로딩 완료")
print(f"API Key 존재: {bool(NOTION_API_KEY)}")
print(f"Database ID 존재: {bool(DATA_SOURCE_ID)}")

✅ 변수 로딩 완료
API Key 존재: True
Database ID 존재: True


In [8]:
# 2. Notion API 기본 설정 (최신 버전 사용)
headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2025-09-03"  # 안정 버전 사용 2022-06-28
}

print("✅ 헤더 설정 완료")
print(f"API Version: {headers['Notion-Version']}")

✅ 헤더 설정 완료
API Version: 2025-09-03


In [16]:
# 6. 페이지 생성 (실제 스키마에 맞춤)
def create_page(page_data):
    """Data Source에 페이지 생성"""
    url = "https://api.notion.com/v1/pages"
    
    payload = {
        "parent": {
            "type": "data_source_id",
            "data_source_id": DATA_SOURCE_ID
        },
        "properties": {
            "company_name": {  # title 타입
                "title": [{"text": {"content": page_data.get("company_name", "")}}]
            },
            "position": {  # rich_text 타입
                "rich_text": [{"text": {"content": page_data.get("position", "")}}]
            },
            "status": {  # select 타입
                "select": {"name": page_data.get("status", "대기중")}
            },
            "apply_date": {  # date 타입
                "date": {"start": page_data.get("apply_date")} if page_data.get("apply_date") else None
            },
            "자소서": {  # checkbox 타입
                "checkbox": page_data.get("자소서", False)
            }
        }
    }
    
    # None 값 제거
    payload["properties"] = {k: v for k, v in payload["properties"].items() if v is not None}
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        print("✅ 페이지 생성 성공!")
        result = response.json()
        page_url = result.get('url', '')
        print(f"📄 생성된 페이지: {page_url}")
        return result
    else:
        print(f"❌ 오류 발생: {response.status_code}")
        error = response.json()
        print(json.dumps(error, indent=2, ensure_ascii=False))
        return None

# 테스트 데이터 (실제 스키마에 맞춤)
test_data = {
    "company_name": "테스트 회사",
    "position": "백엔드 개발자",
    "status": "대기중",
    "apply_date": "2025-12-04",  # YYYY-MM-DD 형식
    "자소서": False
}

print("📝 테스트 데이터:")
print(json.dumps(test_data, indent=2, ensure_ascii=False))
print("\n⚠️  create_page(test_data)를 호출하면 실제로 생성됩니다")

# 주석 해제하면 실제로 생성
result = create_page(test_data)

📝 테스트 데이터:
{
  "company_name": "테스트 회사",
  "position": "백엔드 개발자",
  "status": "대기중",
  "apply_date": "2025-12-04",
  "자소서": false
}

⚠️  create_page(test_data)를 호출하면 실제로 생성됩니다
✅ 페이지 생성 성공!
📄 생성된 페이지: https://www.notion.so/2bf7607904a181169709e5c8e3a37e11


In [17]:
# 7. JD 추가 함수 (실제 스키마에 맞춤)
def add_jd_to_notion(jd_data):
    """JD 분석 결과를 Notion에 추가"""
    url = "https://api.notion.com/v1/pages"
    
    payload = {
        "parent": {
            "type": "data_source_id",
            "data_source_id": DATA_SOURCE_ID
        },
        "properties": {
            "company_name": {
                "title": [{"text": {"content": jd_data.get("company", "")}}]
            },
            "position": {
                "rich_text": [{"text": {"content": jd_data.get("position", "")}}]
            },
            "status": {
                "select": {"name": "대기중"}
            },
            "자소서": {
                "checkbox": False
            }
        }
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        print("✅ JD 추가 성공!")
        result = response.json()
        print(f"📄 {result.get('url', '')}")
        return result
    else:
        print(f"❌ 오류 발생: {response.status_code}")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
        return None

# 테스트 JD (간단 버전)
test_jd = {
    "company": "카카오",
    "position": "AI 엔지니어"
}

print("📝 테스트 JD:")
print(json.dumps(test_jd, indent=2, ensure_ascii=False))
print("\n⚠️  add_jd_to_notion(test_jd)를 호출하면 실제로 추가됩니다")

# 주석 해제하면 실제로 추가
result = add_jd_to_notion(test_jd)

📝 테스트 JD:
{
  "company": "카카오",
  "position": "AI 엔지니어"
}

⚠️  add_jd_to_notion(test_jd)를 호출하면 실제로 추가됩니다
✅ JD 추가 성공!
📄 https://www.notion.so/2bf7607904a18191b59fd839dc831caa


In [19]:
def create_page_with_content(page_data, content_blocks=None):
    """Data Source에 페이지 생성 + 내부 콘텐츠 추가"""
    url = "https://api.notion.com/v1/pages"
    
    payload = {
        "parent": {
            "type": "data_source_id",
            "data_source_id": DATA_SOURCE_ID
        },
        "properties": {
            "company_name": {
                "title": [{"text": {"content": page_data.get("company_name", "")}}]
            },
            "position": {
                "rich_text": [{"text": {"content": page_data.get("position", "")}}]
            },
            "status": {
                "select": {"name": page_data.get("status", "대기중")}
            }
        }
    }
    
    # 콘텐츠 블록 추가
    if content_blocks:
        payload["children"] = content_blocks
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        print("✅ 페이지 생성 성공!")
        result = response.json()
        print(f"📄 {result.get('url', '')}")
        return result
    else:
        print(f"❌ 오류: {response.status_code}")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
        return None

# 테스트: 페이지 생성 + 콘텐츠
test_data = {
    "company_name": "카카오",
    "position": "백엔드 개발자",
    "status": "대기중"
}

# 페이지 내부에 추가할 블록들
content_blocks = [
    {
        "object": "block",
        "type": "heading_2",
        "heading_2": {
            "rich_text": [{"type": "text", "text": {"content": "📋 채용 공고 상세"}}]
        }
    },
    {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
            "rich_text": [
                {"type": "text", "text": {"content": "회사: 카카오\n"}}
            ]
        }
    },
    {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
            "rich_text": [
                {"type": "text", "text": {"content": "포지션: 백엔드 개발자\n"}}
            ]
        }
    },
    {
        "object": "block",
        "type": "heading_3",
        "heading_3": {
            "rich_text": [{"type": "text", "text": {"content": "필수 스킬"}}]
        }
    },
    {
        "object": "block",
        "type": "bulleted_list_item",
        "bulleted_list_item": {
            "rich_text": [{"type": "text", "text": {"content": "Python"}}]
        }
    },
    {
        "object": "block",
        "type": "bulleted_list_item",
        "bulleted_list_item": {
            "rich_text": [{"type": "text", "text": {"content": "Django"}}]
        }
    },
    {
        "object": "block",
        "type": "to_do",
        "to_do": {
            "rich_text": [{"type": "text", "text": {"content": "자소서 작성하기"}}],
            "checked": False
        }
    }
]

# 주석 해제하면 실행
result = create_page_with_content(test_data, content_blocks)

✅ 페이지 생성 성공!
📄 https://www.notion.so/2bf7607904a181e49843f126578926df
